In [128]:
# Import Google Earth Engine.
import ee
# Import Geemap for map visualization.
import geemap.core as geemap

In [129]:
# Authenticate the google earth engine.
ee.Authenticate()

True

In [130]:
# Initialize the GEE.
ee.Initialize()

In [4]:
import ee
import geemap

# Authenticate and initialize Earth Engine
ee.Initialize()

# Define New York City's approximate bounding box as the region of interest (ROI)
nyc_roi = ee.Geometry.Rectangle([-74.2591, 40.4774, -73.7004, 40.9176])  # Coordinates for NYC

# Function to mask clouds using the Sentinel-2 QA band
def maskS2clouds(image):
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    # Return the masked image, divided by 10000 for scaling reflectance values
    return image.updateMask(mask).divide(10000)

# Load Sentinel-2 Harmonized image collection and apply the cloud masking function
dataset = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
              .filterDate('2022-01-01', '2022-01-31') \
              .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
              .map(maskS2clouds)

# Create a median composite image
rgb_image = dataset.median().select(['B4', 'B3', 'B2'])

# Visualization parameters
rgbVis = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2']
}

# Get high-resolution image URL for the defined region (New York City)
url = rgb_image.getThumbURL({
    'region': nyc_roi,
    'dimensions': 1024,  # Increase dimensions for higher resolution
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
    'format': 'png'
})

# Print the high-resolution image URL
print('High-resolution image URL:', url)

# Display the map centered on New York City in Jupyter Notebook
Map = geemap.Map(center=[40.7128, -74.0060], zoom=20)
Map.addLayer(rgb_image, rgbVis, 'Sentinel-2 High-Res RGB')
Map.addLayerControl()
Map


High-resolution image URL: https://earthengine.googleapis.com/v1/projects/428679548430/thumbnails/42e02a4b162ca20777efa4b82e28940e-c9c97e1eb19877808ddbc862d855abd1:getPixels


Map(center=[40.7128, -74.006], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…

In [90]:
# Coordinates for the bounds of a rectangle.
lon_min = -122.09;
lat_min = 37.42;
lon_max = -122.08;
lat_max = 37.43;

In [91]:
# Define an area of interest
selected_region = ee.Geometry.Rectangle([lon_min, lat_min, lon_max, lat_max])

In [118]:
# Test the installation.
print(ee.String('Hello from the Earth Engine servers!').getInfo())

Hello from the Earth Engine servers!


In [120]:
# Function to mask clouds using the QA60 band
def mask_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)


In [132]:
# Load Sentinel-2 Harmonized image collection and apply the cloud masking function
dataset = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
              .filterDate('2022-01-01', '2022-01-31') \
              .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
              .map(mask_clouds)

In [122]:
# Apply cloud masking to the image collection
sentinel2_cloud_free = sentinel2.map(mask_clouds)

In [133]:
# Visualization parameters
rgbVis = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2']
}

In [134]:
# Create a map centered on the desired location
Map = geemap.Map(center=[38.6917, -9.1695], zoom=12)


In [135]:
# Add the dataset's median composite to the map
Map.addLayer(dataset.median(), rgbVis, 'RGB')

In [137]:
# Display the map with layer controls

Map

Map(center=[38.6917, -9.1695], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

In [126]:
# Add Sentinel-2 RGB layer to the map
map.addLayer(rgb_image, vis_params, 'Sentinel-2 RGB')

In [127]:
# Display the map
map

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

In [11]:
# Load a landsat image and select three bands.
landsat = ee.Image('LANDSAT/LC08/C02/T1_TOA/LC08_123032_20140515').select(
    ['B4', 'B3', 'B2']
)

In [117]:
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)


dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate('2020-06-01', '2020-06-30')
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
    .map(mask_s2_clouds)
)

visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

m = geemap.Map()
m.set_center(83.277, 17.7009, 12)
m.add_layer(dataset.mean(), visualization, 'RGB')
m

Map(center=[17.7009, 83.277], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…